# Step 1: Install Required Libraries

In [ ]:
# pip install transformers datasets torch scikit-learn

# Step 2: Prepare Your Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Load your dataset
df = pd.read_csv('spam_data.csv')  # Your dataset with 'text' and 'label' columns

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)


# Step 3: Load and Preprocess the Dataset

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_df = train_df.to_dict(orient='records')
test_df = test_df.to_dict(orient='records')

train_encodings = tokenizer([x['text'] for x in train_df], truncation=True, padding=True, max_length=128)
test_encodings = tokenizer([x['text'] for x in test_df], truncation=True, padding=True, max_length=128)

# Convert labels to tensors
import torch

train_labels = torch.tensor([x['label'] for x in train_df])
test_labels = torch.tensor([x['label'] for x in test_df])

# Create TensorDatasets
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), train_labels)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']), torch.tensor(test_encodings['attention_mask']), test_labels)

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=16)


# Step 4: Load and Fine-Tune the BERT Model

In [ ]:
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup

# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * 4  # Assuming 4 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Move model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Step 5: Training Loop

In [ ]:
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

def train_model(model, train_dataloader, optimizer, scheduler, num_epochs=4):
    model.train()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        for batch in tqdm(train_dataloader):
            b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

            optimizer.zero_grad()

            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()

        print(f"Training loss: {loss.item()}")

train_model(model, train_dataloader, optimizer, scheduler)


# Step 6: Evaluate the Model

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

def evaluate_model(model, test_dataloader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

            outputs = model(b_input_ids, attention_mask=b_input_mask)
            logits = outputs.logits

            predictions.append(logits.argmax(dim=-1).cpu().numpy())
            true_labels.append(b_labels.cpu().numpy())

    predictions = [item for sublist in predictions for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]

    print("Accuracy:", accuracy_score(true_labels, predictions))
    print("Classification Report:\n", classification_report(true_labels, predictions))

evaluate_model(model, test_dataloader)
